In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

# $v_\text{1kpc}$ vs. $v_\text{max}$

First, imports:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import h5py
from astropy import units
from pathlib import Path
import os

import snapshot_obj
import dataset_compute
import curve_fit

import importlib

In [ ]:
importlib.reload(snapshot_obj)
importlib.reload(dataset_compute)
importlib.reload(curve_fit)

## Get data

Next, I retrieve the data. We are only interested in luminous halos. Also, there might be some spurious cases with odd values of $v_\text{max}$.

In [ ]:
snap_id = 127
sim_ids = ["V1_LR_fix", "V1_LR_curvaton_p082_fix", "V1_LR_curvaton_p084_fix"]
names = ["LCDM", "p082", "p084"]
data = {name : {} for name in names}
for sim_id, name in zip(sim_ids, names):
    # Get data:
    snap = snapshot_obj.Snapshot(sim_id, snap_id, name)
    max_point = snap.get_subhalos("Max_Vcirc", "Extended")
    vmax = max_point[:,0] * units.cm.to(units.km)
    rmax = max_point[:,1] * units.cm.to(units.kpc)
    v1kpc = snap.get_subhalos("V1kpc", "Extended") * units.cm.to(units.km)
    
    # Split into satellites:
    vmax_split = dataset_compute.split_subhalos_distict1(snap, vmax)
    v1kpc_split = dataset_compute.split_subhalos_distict1(snap, v1kpc)
    
    # Add to dictionary:
    data[name]["Vmax"] = vmax_split
    data[name]["V1kpc"] = v1kpc_split

## Plot

In [ ]:
# Set some parameters:
x_down = 10; x_up = 100
y_down = 10; y_up = 70

# Set marker styles:
fcolor = ["black", "red", "blue", "green"]
mcolor = ["gray", "pink", "lightblue", "lightgreen"]
marker = ['+', "o", "^", 1]

In [ ]:
# Construct saving location:
filename = 'V1kpc_vs_Vmax'
for name in names:
    filename += "_{}".format(name)
filename += ".png"
    
home = os.path.dirname(snapshot_obj.__file__)
path = os.path.join(home,"Figures")
filename = os.path.join(path, filename)

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(14,6))
plt.subplots_adjust(wspace=0.3)

# Set axis:
for ax in axes:
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlim(x_down, x_up)
    ax.set_ylim(y_down, y_up)
    ax.set_xlabel('$v_{\mathrm{max}} [\mathrm{kms^{-1}}]$', fontsize=16)
    ax.set_ylabel('$v_{\mathrm{1 kpc}} [\mathrm{kms^{-1}}]$', fontsize=16)

axes[0].set_title('Satellites')
axes[1].set_title('Isolated galaxies')

# Add scatter plots:
for i, (name, entry) in enumerate(data.items()):
    x = entry['Vmax']['satellites']
    y = entry['V1kpc']['satellites']
    axes[0].scatter(x, y, s=20, marker=marker[i], c=mcolor[i], \
        edgecolor='none', label=name)
    
    x = entry['Vmax']['isolated']
    y = entry['V1kpc']['isolated']
    axes[1].scatter(x, y, s=20, marker=marker[i], c=mcolor[i], \
        edgecolor='none', label=name)
    
# Add median curves:
for i, (name, entry) in enumerate(data.items()):
    x = entry['Vmax']['satellites']
    y = entry['V1kpc']['satellites']    
    median = curve_fit.median_trend(x, y, points_per_bar=7)
    axes[0].plot(median[0], median[1], c=fcolor[i], linestyle='--')
    
    x = entry['Vmax']['isolated']
    y = entry['V1kpc']['isolated'] 
    median = curve_fit.median_trend(x, y, points_per_bar=7)
    axes[1].plot(median[0], median[1], c=fcolor[i], linestyle='--')
    
axes[0].legend(loc='lower right')
plt.tight_layout()

#plt.savefig(filename, dpi=200)